# Warsztat 3: Widoki i Podstawowe Jobsy

**Czas trwania**: 90 minut  
**Poziom**: Podstawowy/Średniozaawansowany  
**Ostatnia aktualizacja**: 2025-01-31

---

## 1. Kontekst i Cele Warsztatu

### Czego się nauczysz?
- Tworzyć widoki tymczasowe i globalne
- Tworzyć widoki materialized w Delta Lake
- Konfigurować podstawowe joby w Databricks
- Monitorować wykonanie jobów

### Wymagania wstępne
- Ukończenie Warsztatu 1 i 2
- Podstawowa znajomość SQL i PySpark
- Wyczyszczone dane z poprzedniego warsztatu

### Co będziesz implementować?
Stworzysz zestaw widoków biznesowych i skonfigurujesz job ETL, który automatycznie przetwarza dane.

---

## 2. Materiały Teoretyczne

### 2.1 Typy Widoków
- **Temporary View** - widok tymczasowy, dostępny tylko w bieżącej sesji Spark
- **Global Temporary View** - widok globalny, dostępny we wszystkich sesjach Spark
- **View (Materialized)** - widok trwały, zapisany w metastore

### 2.2 Databricks Jobs
- **Task** - pojedyncze zadanie (notebook, JAR, Python script)
- **Job** - zestaw tasków z zależnościami
- **Cluster** - zasoby obliczeniowe do wykonania jobu
- **Schedule** - harmonogram uruchamiania jobu

### 2.3 Best Practices
- Używaj widoków dla często używanych zapytań
- Dokumentuj widoki - dodawaj komentarze
- Monitoruj performance jobów
- Używaj retry policy dla jobów

---

## 3. Inicjalizacja Środowiska

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [ ]:
%run ../../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

### 3.2 Konfiguracja Zmiennych

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Nazwy tabel z poprzednich warsztatów
CUSTOMERS_TABLE = f"{SILVER_SCHEMA}.customers_clean"
ORDERS_TABLE = f"{BRONZE_SCHEMA}.orders_workshop"
PRODUCTS_TABLE = f"{BRONZE_SCHEMA}.products_workshop"

# Nazwy widoków do utworzenia
CUSTOMER_ORDERS_VIEW = f"{SILVER_SCHEMA}.vw_customer_orders"
TOP_CUSTOMERS_VIEW = f"{SILVER_SCHEMA}.vw_top_customers"
PRODUCT_SALES_VIEW = f"{SILVER_SCHEMA}.vw_product_sales"

**Wyjaśnienie konfiguracji:**
- Importujemy biblioteki PySpark do transformacji danych
- Definiujemy nazwy tabel źródłowych z poprzednich warsztatów
- Konfigurujemy nazwy widoków, które będziemy tworzyć w tym warsztacie
- Widoki będą przechowywane w schemacie SILVER dla czytelności i organizacji

---

## 4. Widoki Tymczasowe

### 4.1 Przykład: Temporary View
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Tworzenie widoku tymczasowego
# Widok będzie dostępny tylko w tej sesji Spark

customers_df = spark.table(CUSTOMERS_TABLE)
customers_df.createOrReplaceTempView("tmp_customers")

# Użyj widoku w zapytaniu SQL
result = spark.sql("""
    SELECT 
        city,
        COUNT(*) as customer_count
    FROM tmp_customers
    GROUP BY city
    ORDER BY customer_count DESC
    LIMIT 10
""")

display(result)

**Wyjaśnienie Temporary View:**
- Widok tymczasowy `tmp_customers` istnieje tylko w tej sesji Spark
- Pozwala na używanie SQL na DataFrame bez zapisywania do metastore
- Idealne do eksploracji danych i tymczasowych analiz
- Query pokazuje top 10 miast z największą liczbą klientów

### 4.2 Zadanie: Global Temporary View

**TODO**: Uzupełnij kod, aby:
1. Utworzyć globalny widok tymczasowy dla tabeli zamówień
2. Wyświetlić statystyki zamówień z tego widoku

**Wskazówka**: Użyj `.createOrReplaceGlobalTempView()` i odwołuj się przez `global_temp.nazwa_widoku`

In [ ]:
# TODO: Utwórz globalny widok tymczasowy
orders_df = spark.table(ORDERS_TABLE)

# TODO: Uzupełnij kod
# orders_df.createOrReplaceGlobalTempView("____")

# TODO: Użyj widoku w zapytaniu SQL
# Wskazówka: SELECT ... FROM global_temp.gtmp_orders
result = spark.sql("""
    -- TODO: Napisz zapytanie, które policzy:
    -- - Całkowitą liczbę zamówień: COUNT(order_id)
    -- - Całkowitą wartość zamówień: SUM(total_amount)
    -- - Średnią wartość zamówienia: AVG(total_amount)
    -- - Minimalną i maksymalną wartość: MIN/MAX(total_amount)
    SELECT 
        COUNT(____) as total_orders,
        SUM(____) as total_value,
        AVG(____) as avg_order_value,
        MIN(____) as min_order_value,
        MAX(____) as max_order_value
    FROM global_temp.____
""")

display(result)

**Wyjaśnienie Global Temporary View:**
- Global Temporary View jest dostępny we wszystkich sesjach Spark w tym samym aplikacji
- Używa prefiksu `global_temp.` w zapytaniach SQL
- Przydatny do współdzielenia danych między różnymi notebook'ami
- Kolumny w tabeli orders: `order_id`, `total_amount`, `quantity`, `payment_method`

---

## 5. Widoki Materialized (Trwałe)

### 5.1 Przykład: Widok Zamówień z Klientami
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Widok łączący zamówienia z klientami
spark.sql(f"""
    CREATE OR REPLACE VIEW {CUSTOMER_ORDERS_VIEW}
    COMMENT 'Widok łączący dane zamówień z informacjami o klientach'
    AS
    SELECT 
        o.order_id,
        o.order_datetime,
        o.total_amount,
        o.payment_method,
        o.quantity,
        c.customer_id,
        c.first_name,
        c.last_name,
        c.email,
        c.city,
        c.country,
        c.customer_segment
    FROM {ORDERS_TABLE} o
    INNER JOIN {CUSTOMERS_TABLE} c
        ON o.customer_id = c.customer_id
""")

# Sprawdź widok
result = spark.sql(f"SELECT * FROM {CUSTOMER_ORDERS_VIEW} LIMIT 10")
display(result)

**Wyjaśnienie Materialized View:**
- Widok zostaje zapisany w metastore i jest dostępny dla wszystkich użytkowników
- JOIN łączy tabele zamówień i klientów po `customer_id`
- Zawiera kompletne informacje o zamówieniach wraz z danymi klientów
- Użyte kolumny to rzeczywiste kolumny z dataset'u: `order_datetime`, `total_amount`, `payment_method`, `customer_segment`

### 5.2 Zadanie: Widok Top Klientów

**TODO**: Uzupełnij kod, aby utworzyć widok, który:
1. Policzy liczbę zamówień dla każdego klienta
2. Obliczy całkowitą wartość zamówień dla każdego klienta
3. Obliczy średnią wartość zamówienia
4. Pokaże tylko top 100 klientów (według wartości)

**Wskazówka**: Użyj `CREATE OR REPLACE VIEW` z `GROUP BY` i `ORDER BY ... LIMIT`

In [ ]:
# TODO: Utwórz widok top klientów
spark.sql(f"""
    CREATE OR REPLACE VIEW {TOP_CUSTOMERS_VIEW}
    COMMENT 'Top 100 klientów według wartości zamówień'
    AS
    SELECT 
        customer_id,
        first_name,
        last_name,
        email,
        city,
        customer_segment,
        COUNT(____) AS order_count,
        SUM(____) AS total_spent,
        AVG(____) AS avg_order_value
    FROM {CUSTOMER_ORDERS_VIEW}
    GROUP BY customer_id, first_name, last_name, email, city, customer_segment
    ORDER BY ____ DESC
    LIMIT ____
""")

print(f"Utworzono widok: {TOP_CUSTOMERS_VIEW}")

# Sprawdź widok
result = spark.sql(f"SELECT * FROM {TOP_CUSTOMERS_VIEW} LIMIT 10")
display(result)

**Wskazówki do uzupełnienia:**
- `COUNT(____)` - policz `order_id` aby uzyskać liczbę zamówień
- `SUM(____)` - zsumuj `total_amount` aby uzyskać całkowitą wartość
- `AVG(____)` - uśrednij `total_amount` dla średniej wartości zamówienia
- `ORDER BY ____` - sortuj według `total_spent` malejąco
- `LIMIT ____` - ogranicz do 100 rekordów

### 5.3 Zadanie: Widok Sprzedaży Produktów

**TODO**: Uzupełnij kod, aby utworzyć widok sprzedaży produktów.
Zakładamy, że tabela zamówień ma kolumnę `product_id` i `quantity`.

**Wskazówka**: Połącz tabele zamówień i produktów, oblicz statystyki sprzedaży

In [ ]:
# TODO: Utwórz widok sprzedaży produktów
spark.sql(f"""
    CREATE OR REPLACE VIEW {PRODUCT_SALES_VIEW}
    COMMENT 'Statystyki sprzedaży produktów'
    AS
    SELECT 
        p.product_id,
        p.product_name,
        p.category,
        p.price,
        COUNT(o.order_id) AS order_count,
        SUM(o.____) AS total_quantity,
        SUM(o.____) AS total_revenue,
        AVG(o.____) AS avg_order_value
    FROM {ORDERS_TABLE} o
    INNER JOIN {PRODUCTS_TABLE} p
        ON o.____ = p.____
    GROUP BY p.product_id, p.product_name, p.category, p.price
    ORDER BY ____ DESC
""")

print(f"Utworzono widok: {PRODUCT_SALES_VIEW}")

# Sprawdź widok
result = spark.sql(f"SELECT * FROM {PRODUCT_SALES_VIEW} LIMIT 10")
display(result)

**Wskazówki do uzupełnienia Product Sales:**
- `SUM(o.____)` - zsumuj `quantity` i `total_amount` dla statystyk
- `AVG(o.____)` - uśrednij `total_amount` dla średniej wartości
- `ON o.____ = p.____` - połącz tabele przez `product_id`
- `ORDER BY ____` - sortuj według `total_revenue` malejąco
- Kolumny produktów: `product_name`, `category`, `price`

---

## 6. Zapytania na Widokach

### 6.1 Przykład: Analiza Top Klientów
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Zapytanie na widoku top klientów
result = spark.sql(f"""
    SELECT 
        customer_id,
        first_name || ' ' || last_name AS customer_name,
        city,
        customer_segment,
        order_count,
        ROUND(total_spent, 2) AS total_spent,
        ROUND(avg_order_value, 2) AS avg_order_value
    FROM {TOP_CUSTOMERS_VIEW}
    WHERE order_count >= 5
    ORDER BY total_spent DESC
    LIMIT 20
""")

display(result)

**Wyjaśnienie zapytania na widoku:**
- Wykorzystujemy wcześniej utworzony widok `vw_top_customers`
- Łączymy imię i nazwisko w jedną kolumnę `customer_name`
- Filtrujemy klientów z minimum 5 zamówieniami
- Zaokrąglamy wartości finansowe do 2 miejsc po przecinku
- Sortujemy według całkowitej wartości zamówień

### 6.2 Zadanie: Analiza Segmentów Klientów

**TODO**: Napisz zapytanie SQL, które:
1. Użyje widoku `TOP_CUSTOMERS_VIEW`
2. Podzieli klientów na segmenty:
   - VIP: total_spent >= 5000
   - Premium: total_spent >= 2000 AND < 5000
   - Regular: total_spent < 2000
3. Policzy liczbę klientów w każdym segmencie
4. Obliczy średnią wartość zamówienia dla każdego segmentu

**Wskazówka**: Użyj `CASE WHEN` do utworzenia segmentów

In [ ]:
# TODO: Uzupełnij zapytanie SQL
result = spark.sql(f"""
    SELECT 
        CASE 
            WHEN total_spent >= ____ THEN 'VIP'
            WHEN total_spent >= ____ THEN 'Premium'
            ELSE 'Regular'
        END AS segment,
        COUNT(*) AS customer_count,
        AVG(____) AS avg_order_value,
        SUM(____) AS total_revenue,
        ROUND(AVG(total_spent), 2) AS avg_customer_value
    FROM {TOP_CUSTOMERS_VIEW}
    GROUP BY ____
    ORDER BY total_revenue ____
""")

display(result)

**Wskazówki do segmentacji klientów:**
- `WHEN total_spent >= ____` - ustaw próg na 5000 dla VIP, 2000 dla Premium
- `AVG(____)` - użyj `avg_order_value` do obliczenia średniej dla segmentu
- `SUM(____)` - użyj `total_spent` do obliczenia łącznego przychodu
- `GROUP BY ____` - grupuj według `segment`
- `ORDER BY total_revenue ____` - sortuj malejąco (`DESC`)

---

## 7. Zarządzanie Widokami

### 7.1 Przykład: Sprawdzanie Metadanych Widoku
Ta komórka jest już gotowa - przeanalizuj kod i uruchom go.

In [ ]:
# Przykład: Sprawdzenie metadanych widoku
spark.sql(f"DESCRIBE EXTENDED {CUSTOMER_ORDERS_VIEW}").show(truncate=False)

spark.sql(f"SHOW CREATE TABLE {CUSTOMER_ORDERS_VIEW}").show(truncate=False)

**Wyjaśnienie metadanych widoku:**
- `DESCRIBE EXTENDED` pokazuje szczegóły struktury widoku, typy kolumn i metadane
- `SHOW CREATE TABLE` wyświetla definicję SQL użytą do utworzenia widoku
- Przydatne do debugowania i dokumentacji widoków
- Pozwala sprawdzić czy widok został poprawnie utworzony

### 7.2 Zadanie: Lista Wszystkich Widoków

**TODO**: Uzupełnij kod, aby:
1. Wyświetlić listę wszystkich widoków w schemacie SILVER
2. Dla każdego widoku wyświetlić: nazwę, typ (VIEW vs TABLE), komentarz

**Wskazówka**: Użyj `SHOW TABLES IN schema` i przefiltruj po typie

In [ ]:
# TODO: Wyświetl listę widoków
# Wskazówka: SHOW TABLES IN schema
result = spark.sql(f"""
    SHOW TABLES IN {____}
""")

# Przefiltruj tylko widoki (isTemporary = false)
views_only = result.filter(F.col("isTemporary") == ____)

display(views_only)

**Wskazówki do listowania widoków:**
- `SHOW TABLES IN {____}` - użyj `SILVER_SCHEMA` 
- `F.col("isTemporary") == ____` - użyj `False` aby pokazać tylko trwałe widoki
- Komenda `SHOW TABLES` pokazuje zarówno tabele jak i widoki
- Kolumna `isTemporary` pozwala odfiltrować widoki tymczasowe

---

## 8. Przygotowanie Notebooka dla Joba

### 8.1 Przykład: Parametryzowany Notebook
Ta komórka demonstruje, jak utworzyć notebook, który może być uruchomiony jako job z parametrami.

In [ ]:
# Przykład: Widget do parametryzacji notebooka
# W Databricks możesz utworzyć widget, który pozwala przekazać parametry do notebooka

# Utwórz widget dla daty przetwarzania
dbutils.widgets.text("processing_date", "2025-01-31", "Data przetwarzania")
dbutils.widgets.dropdown("mode", "full", ["full", "incremental"], "Tryb przetwarzania")

# Odczytaj wartości parametrów
processing_date = dbutils.widgets.get("processing_date")
mode = dbutils.widgets.get("mode")

**Wyjaśnienie parametryzacji notebooka:**
- `dbutils.widgets` pozwala utworzyć interfejs do przekazywania parametrów
- Widget tekstowy `processing_date` dla daty przetwarzania
- Widget dropdown `mode` z opcjami: "full" (pełne) lub "incremental" (przyrostowe)
- Parametry mogą być przekazane z Databricks Jobs lub ustawione ręcznie

### 8.2 Zadanie: ETL Pipeline

**TODO**: Uzupełnij kod pipeline'u, który:
1. Wczyta dane z tabel bronze
2. Wyczyści i przekształci dane
3. Zapisze wyniki do tabel silver
4. Odświeży widoki
5. Wyświetli statystyki przetwarzania

Ten kod będzie używany w jobie Databricks.

In [ ]:
# TODO: Implementuj pipeline ETL - Krok 1: Inicjalizacja
from datetime import datetime

# Rozpocznij pipeline
start_time = datetime.now()
print(f"=== Pipeline ETL rozpoczęty: {start_time} ===")

# Krok 1: Wczytaj dane bronze
print("\n[1/5] Wczytywanie danych bronze...")
# TODO: Wczytaj tabele bronze
customers_bronze = spark.table(f"{BRONZE_SCHEMA}.customers_workshop")
orders_bronze = spark.table(f"{BRONZE_SCHEMA}.orders_workshop")

print(f"  - Klienci: {customers_bronze.count()} wierszy")
print(f"  - Zamówienia: {orders_bronze.count()} wierszy")

# Krok 2: Czyszczenie danych
print("\n[2/5] Czyszczenie danych...")
# TODO: Usuń duplikaty, obsłuż NULL, standaryzuj dane
customers_clean = (
    customers_bronze
    .dropDuplicates(["customer_id"])
    .withColumn("email", F.trim(F.lower(F.col("email"))))
    .withColumn("last_name", F.initcap(F.col("last_name")))
)

orders_clean = (
    orders_bronze
    .dropna(subset=["order_amount"])
    .fillna({"order_status": "Unknown"})
)

print(f"  - Klienci po czyszczeniu: {customers_clean.count()} wierszy")
print(f"  - Zamówienia po czyszczeniu: {orders_clean.count()} wierszy")

# Krok 3: Zapis do silver
print("\n[3/5] Zapis do silver schema...")
# TODO: Zapisz wyczyszczone dane do silver
(
    customers_clean
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{SILVER_SCHEMA}.customers_clean")
)

(
    orders_clean
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{SILVER_SCHEMA}.orders_clean")
)

print("  - Tabele zapisane do silver")

# Krok 4: Odśwież widoki (opcjonalnie recreate)
print("\n[4/5] Odświeżanie widoków...")
# TODO: Odśwież widoki jeśli to konieczne
# Wskazówka: W Delta Lake widoki są automatycznie aktualizowane
print("  - Widoki odświeżone")

# Krok 5: Statystyki
print("\n[5/5] Statystyki przetwarzania...")
end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

print(f"\n=== Pipeline ETL zakończony ===")
print(f"Czas rozpoczęcia: {start_time}")
print(f"Czas zakończenia: {end_time}")
print(f"Czas trwania: {duration} sekund")
print(f"\nPrzetworzone rekordy:")
print(f"  - Klienci: {customers_clean.count()}")
print(f"  - Zamówienia: {orders_clean.count()}")

**Krok 9: Finalizacja i statystyki**
- Zapisujemy czas zakończenia i obliczamy czas trwania pipeline'u
- Wyświetlamy podsumowanie z kluczowymi metrykami
- To pomaga w monitorowaniu wydajności procesu ETL
- W środowisku produkcyjnym te metryki można logować do systemu monitoringu

In [ ]:
# TODO: Krok 9: Finalizacja i statystyki
end_time = datetime.now()
duration = (end_time - start_time).total_seconds()

# Wyświetl podsumowanie pipeline'u
display(spark.createDataFrame([
    ("Czas rozpoczęcia", str(start_time)),
    ("Czas zakończenia", str(end_time)),
    ("Czas trwania (sekundy)", str(duration)),
    ("Klienci przetworzone", str(customers_clean_count)),
    ("Zamówienia przetworzone", str(orders_clean_count))
], ["Metryka", "Wartość"]))

**Krok 8: Zapis zamówień do Silver**
- Analogicznie jak dla klientów - używamy Delta Lake format
- Zapisujemy wyczyszczone zamówienia do warstwy Silver
- Warstwa Silver zawiera wyczyszczone i zwalidowane dane

**Wskazówki do zadania podsumowującego:**
- `MONTH(____)` - użyj `order_datetime` 
- `COUNT(____)` - policz `order_id`
- `SUM(____)` i `AVG(____)` - użyj `total_amount`
- `COUNT(DISTINCT ____)` - policz unikalnych `customer_id`
- `WHERE ____ IS NOT NULL` - filtruj po `order_datetime`
- `LAG(____)` - użyj `total_sales` do obliczenia trendu MoM (month-over-month)

In [ ]:
# TODO: Krok 8: Zapisz zamówienia do Silver
(
    orders_clean
    .write
    .format("____")  # delta
    .mode("____")    # overwrite
    .option("overwriteSchema", "true")
    .saveAsTable(f"{____}.orders_clean")  # SILVER_SCHEMA
)

**Krok 7: Zapis klientów do Silver**
- `.format("____")` - użyj `"delta"` dla Delta Lake
- `.mode("____")` - użyj `"overwrite"` aby zastąpić istniejące dane
- `.saveAsTable(f"{____}.customers_clean")` - użyj `SILVER_SCHEMA`
- `overwriteSchema` pozwala na zmiany w schemacie tabeli

**Czyszczenie zasobów:**
- Opcjonalnie usuwamy utworzone widoki testowe
- Usuwamy widoki tymczasowe z pamięci Spark
- Usuwamy wszystkie widgety utworzone w notebook'u
- W środowisku produkcyjnym należy zachować widoki do dalszego użytku

In [ ]:
# TODO: Krok 7: Zapisz klientów do Silver
(
    customers_clean
    .write
    .format("____")  # delta
    .mode("____")    # overwrite
    .option("overwriteSchema", "true")
    .saveAsTable(f"{____}.customers_clean")  # SILVER_SCHEMA
)

**Krok 6: Sprawdzenie wyników czyszczenia**
- Ponownie używamy `.count()` aby sprawdzić ile rekordów zostało po czyszczeniu
- Porównujemy liczby przed i po czyszczeniu
- To pozwala ocenić jakość danych i skuteczność procesu czyszczenia

In [ ]:
# TODO: Krok 6: Sprawdź wyniki czyszczenia
customers_clean_count = customers_clean.____()
orders_clean_count = orders_clean.____()

# Wyświetl porównanie przed i po czyszczeniu
display(spark.createDataFrame([
    ("Klienci - przed", customers_count),
    ("Klienci - po", customers_clean_count),
    ("Zamówienia - przed", orders_count),
    ("Zamówienia - po", orders_clean_count)
], ["Etap", "Liczba_rekordów"]))

**Krok 5: Czyszczenie danych zamówień**
- `.dropna(subset=["____"])` - użyj `total_amount` aby usunąć zamówienia bez wartości
- `.fillna({"payment_method": "____"})` - użyj `"Unknown"` dla brakujących metod płatności
- Filtrujemy zamówienia z dodatnią wartością `total_amount` i `quantity`
- To usuwa błędne dane z ujemnymi wartościami

In [ ]:
# TODO: Krok 5: Czyszczenie danych zamówień
orders_clean = (
    orders_bronze
    .dropna(subset=["____"])  # usuń rekordy bez total_amount
    .fillna({"payment_method": "____"})  # wypełnij brakujące metody płatności
    .filter(F.col("____") > 0)  # usuń zamówienia z ujemną wartością
    .filter(F.col("____") > 0)  # usuń zamówienia z ujemną ilością
)

**Krok 4: Czyszczenie danych klientów**
- `.dropDuplicates(["____"])` - użyj `customer_id` aby usunąć duplikaty
- Standaryzujemy email: `.trim()` usuwa spacje, `.lower()` zmienia na małe litery
- `.initcap()` - zmienia pierwszą literę na wielką dla `last_name` i `first_name`

In [ ]:
# TODO: Krok 4: Czyszczenie danych klientów
customers_clean = (
    customers_bronze
    .dropDuplicates(["____"])
    .withColumn("email", F.trim(F.lower(F.col("____"))))
    .withColumn("last_name", F.initcap(F.col("____")))
    .withColumn("first_name", F.initcap(F.col("____")))
)

**Krok 3: Sprawdzenie liczby rekordów**
- Użyj metody `count()` do policzenia rekordów w każdej tabeli
- Wyświetlamy statystyki w czytelnej formie tabeli
- To pomaga zweryfikować, że dane zostały poprawnie wczytane

In [ ]:
# TODO: Krok 3: Sprawdź liczby rekordów
customers_count = customers_bronze.____()
orders_count = orders_bronze.____()

# Wyświetl statystyki wczytanych danych
display(spark.createDataFrame([
    ("Klienci", customers_count),
    ("Zamówienia", orders_count)
], ["Tabela", "Liczba_rekordów"]))

**Krok 2: Wczytywanie danych Bronze**
- Wczytujemy tabele z warstwy Bronze (surowe dane)
- Użyj `BRONZE_SCHEMA` w miejscach z `____`
- Tabele `customers_workshop` i `orders_workshop` zostały utworzone w poprzednich warsztatach

In [ ]:
# TODO: Krok 2: Wczytaj dane bronze
customers_bronze = spark.table(f"{____}.customers_workshop")
orders_bronze = spark.table(f"{____}.orders_workshop")

**Krok 1: Inicjalizacja Pipeline'u ETL**
- Importujemy `datetime` do śledzenia czasu wykonania
- Zapisujemy czas rozpoczęcia pipeline'u
- To pozwoli nam zmierzyć wydajność procesu ETL

---

## 9. Konfiguracja Joba (Instrukcja)

### 9.1 Tworzenie Joba w Databricks UI

**Krok po kroku:**

1. **Przejdź do zakładki Jobs**
   - W menu bocznym wybierz "Workflows" → "Jobs"
   - Kliknij "Create Job"

2. **Skonfiguruj Task**
   - **Task name**: `etl_silver_pipeline`
   - **Type**: `Notebook`
   - **Source**: Wybierz ten notebook
   - **Cluster**: Wybierz istniejący cluster lub utwórz nowy

3. **Parametry (opcjonalnie)**
   - Dodaj parametry dla `processing_date` i `mode`
   - Przykład: `{"processing_date": "2025-01-31", "mode": "full"}`

4. **Harmonogram (opcjonalnie)**
   - **Schedule**: Cron expression, np. `0 2 * * *` (codziennie o 2:00)
   - **Timezone**: Wybierz swoją strefę czasową

5. **Retry Policy**
   - **Max retries**: 3
   - **Retry interval**: 60 sekund

6. **Notifications (opcjonalnie)**
   - Dodaj email dla powiadomień o sukcesie/błędzie

7. **Zapisz i uruchom**
   - Kliknij "Create" aby zapisać job
   - Kliknij "Run now" aby uruchomić job ręcznie

### 9.2 Monitorowanie Joba

Po uruchomieniu joba:
- Obserwuj status w zakładce "Runs"
- Sprawdź logi w "Logs"
- Analizuj metryki w "Metrics"
- Sprawdź outputy notebooka

---

## 10. Zadanie Podsumowujące

### 10.1 TODO: Kompleksowe Zadanie

**Zadanie**: Zaimplementuj kompletny proces:

1. Utwórz widok `vw_monthly_sales` pokazujący:
   - Rok i miesiąc
   - Liczbę zamówień
   - Całkowitą wartość sprzedaży
   - Średnią wartość zamówienia
   - Liczbę unikalnych klientów

2. Napisz zapytanie, które:
   - Użyje utworzonego widoku
   - Obliczy wzrost sprzedaży miesiąc do miesiąca (MoM growth)
   - Wyświetli trendy

3. Przygotuj ten kod do uruchomienia jako job

In [ ]:
# TODO: Implementuj zadanie podsumowujące

# Część 1: Utwórz widok monthly_sales
spark.sql(f"""
    CREATE OR REPLACE VIEW {SILVER_SCHEMA}.vw_monthly_sales
    COMMENT 'Miesięczne statystyki sprzedaży'
    AS
    SELECT 
        YEAR(order_datetime) as year,
        MONTH(____) as month,
        COUNT(____) as order_count,
        SUM(____) as total_sales,
        AVG(____) as avg_order_value,
        COUNT(DISTINCT ____) as unique_customers
    FROM {SILVER_SCHEMA}.orders_clean
    WHERE ____ IS NOT NULL
    GROUP BY YEAR(order_datetime), MONTH(order_datetime)
    ORDER BY year, month
""")

# Część 2: Zapytanie z trendem MoM
result = spark.sql(f"""
    SELECT 
        year,
        month,
        order_count,
        total_sales,
        LAG(____) OVER (ORDER BY year, month) as prev_month_sales,
        ROUND(
            (total_sales - LAG(____) OVER (ORDER BY year, month)) / 
            LAG(____) OVER (ORDER BY year, month) * 100, 2
        ) as mom_growth_percent
    FROM {SILVER_SCHEMA}.vw_monthly_sales
    ORDER BY year, month
""")

display(result)

# Część 3: Dodaj error handling i logging
# TODO: try-except, logging statystyk

print("Zadanie zakończone!")

---

## 11. Best Practices i Podsumowanie

### 11.1 Best Practices dla Widoków
1. **Nazywaj widoki prefiksem** - `vw_` dla łatwej identyfikacji
2. **Dodawaj komentarze** - dokumentuj cel widoku
3. **Używaj widoków dla logiki biznesowej** - enkapsulacja złożonych zapytań
4. **Monitoruj performance** - widoki mogą wpływać na wydajność
5. **Rozważ materialized views** - dla często używanych, kosztownych zapytań

### 11.2 Best Practices dla Jobs
1. **Parametryzuj notebooki** - używaj widgets
2. **Implementuj error handling** - try-except, retry policy
3. **Loguj wszystko** - starty, zakończenia, błędy, statystyki
4. **Monitoruj joby** - alerting, notifications
5. **Testuj przed wdrożeniem** - uruchom ręcznie i sprawdź wyniki
6. **Używaj idempotentnych operacji** - job można uruchomić wielokrotnie bez efektów ubocznych

### 11.3 Co osiągnąłeś?
- Stworzyłeś widoki tymczasowe i trwałe
- Stworzyłeś widoki biznesowe (top klienci, sprzedaż produktów)
- Napisałeś zapytania wykorzystujące widoki
- Przygotowałeś notebook do uruchomienia jako job
- Zaimplementowałeś prosty pipeline ETL
- Poznałeś best practices dla widoków i jobs

### 11.4 Następne kroki
- Delta Live Tables (DLT)
- Zaawansowane joby (multi-task workflows)
- Monitoring i alerting
- CI/CD dla Databricks

---

## 12. Czyszczenie Zasobów

In [ ]:
# Opcjonalnie: usuń widoki testowe
# spark.sql(f"DROP VIEW IF EXISTS {CUSTOMER_ORDERS_VIEW}")
# spark.sql(f"DROP VIEW IF EXISTS {TOP_CUSTOMERS_VIEW}")
# spark.sql(f"DROP VIEW IF EXISTS {PRODUCT_SALES_VIEW}")

# Usuń widoki tymczasowe
# spark.catalog.dropTempView("tmp_customers")
# spark.catalog.dropGlobalTempView("gtmp_orders")

# Usuń widgety
dbutils.widgets.removeAll()

display(spark.createDataFrame([
    ("Status", "Zasoby wyczyszczone!"),
    ("Warsztat", "Zakończony!")
], ["Kategoria", "Informacja"]))